Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [149]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [150]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [37]:
batch_size = 128
hidden_nodes = 1024
learning_rate = 0.5
beta = 0.005

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_1 = tf.Variable(tf.zeros([hidden_nodes]))
  weights_2 = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  def forward_prop(input):
    h1 = tf.nn.relu(tf.matmul(input, weights_1) + biases_1)
    return tf.matmul(h1, weights_2) + biases_2
  
  logits = forward_prop(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  # Add the regularization term to the loss.
  loss += beta * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset))
  test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset))

In [38]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1942.353394
Minibatch accuracy: 7.8%
Validation accuracy: 32.5%
Minibatch loss at step 500: 127.152344
Minibatch accuracy: 79.7%
Validation accuracy: 83.0%
Minibatch loss at step 1000: 11.105256
Minibatch accuracy: 78.1%
Validation accuracy: 84.9%
Minibatch loss at step 1500: 1.552540
Minibatch accuracy: 82.8%
Validation accuracy: 85.2%
Minibatch loss at step 2000: 0.574066
Minibatch accuracy: 90.6%
Validation accuracy: 85.7%
Minibatch loss at step 2500: 0.555076
Minibatch accuracy: 89.1%
Validation accuracy: 85.8%
Minibatch loss at step 3000: 0.633094
Minibatch accuracy: 85.2%
Validation accuracy: 85.2%
Test accuracy: 91.2%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [17]:
train_dataset_restricted = train_dataset[:130, :]
train_labels_restricted = train_labels[:130, :]

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_restricted.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_restricted[offset:(offset + batch_size), :]
    batch_labels = train_labels_restricted[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1893.680908
Minibatch accuracy: 15.6%
Validation accuracy: 26.3%
Minibatch loss at step 500: 128.907288
Minibatch accuracy: 100.0%
Validation accuracy: 66.5%
Minibatch loss at step 1000: 10.581758
Minibatch accuracy: 100.0%
Validation accuracy: 69.9%
Minibatch loss at step 1500: 0.984593
Minibatch accuracy: 100.0%
Validation accuracy: 70.8%
Minibatch loss at step 2000: 0.194832
Minibatch accuracy: 100.0%
Validation accuracy: 71.0%
Minibatch loss at step 2500: 0.123936
Minibatch accuracy: 100.0%
Validation accuracy: 71.2%
Minibatch loss at step 3000: 0.114682
Minibatch accuracy: 100.0%
Validation accuracy: 71.3%
Test accuracy: 77.6%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [170]:
batch_size = 128
hidden_nodes_1 = 1024
hidden_nodes_2 = 512
hidden_nodes_3 = 1024
learning_rate = 0.0001
beta = 0.005

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Placeholder to control dropout probability.
  keep_prob = tf.placeholder(tf.float32)

  # Variables.
  weights_1 = tf.Variable(tf.random_normal([image_size * image_size, hidden_nodes_1]))
  biases_1 = tf.Variable(tf.zeros([hidden_nodes_1]))
  weights_2 = tf.Variable(tf.random_normal([hidden_nodes_1, hidden_nodes_2]))
  biases_2 = tf.Variable(tf.zeros([hidden_nodes_2]))
  weights_3 = tf.Variable(tf.random_normal([hidden_nodes_2, hidden_nodes_3]))
  biases_3 = tf.Variable(tf.zeros([hidden_nodes_3]))
  weights_out = tf.Variable(tf.random_normal([hidden_nodes_3, num_labels]))
  biases_out = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  def forward_prop(input):
    h1 = tf.nn.dropout(tf.nn.relu(tf.matmul(input, weights_1) + biases_1), keep_prob)
    h2 = tf.nn.dropout(tf.nn.relu(tf.matmul(   h1, weights_2) + biases_2), keep_prob)
    h3 = tf.nn.dropout(tf.nn.relu(tf.matmul(   h2, weights_3) + biases_3), keep_prob)
    return tf.matmul(h3, weights_out) + biases_out
  
  logits = forward_prop(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  # Add the regularization term to the loss.
  loss += beta * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(weights_3) + tf.nn.l2_loss(weights_out))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
#  optimizer = tf.train.AdamOptimizer(0.0001).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset))
  test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset))

In [171]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 1.0}
    feed_dict_w_drop = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict_w_drop)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(feed_dict=feed_dict), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict=feed_dict), test_labels))

Initialized
Minibatch loss at step 0: 446566.125000
Minibatch accuracy: 10.9%
Validation accuracy: 14.1%
Minibatch loss at step 500: 30018.687500
Minibatch accuracy: 61.7%
Validation accuracy: 79.3%
Minibatch loss at step 1000: 17007.601562
Minibatch accuracy: 63.3%
Validation accuracy: 80.5%
Minibatch loss at step 1500: 11282.326172
Minibatch accuracy: 67.2%
Validation accuracy: 80.6%
Minibatch loss at step 2000: 10362.789062
Minibatch accuracy: 67.2%
Validation accuracy: 80.5%
Minibatch loss at step 2500: 8313.672852
Minibatch accuracy: 67.2%
Validation accuracy: 80.5%
Minibatch loss at step 3000: 6849.096680
Minibatch accuracy: 64.8%
Validation accuracy: 80.0%
Minibatch loss at step 3500: 6824.620117
Minibatch accuracy: 61.7%
Validation accuracy: 80.1%
Minibatch loss at step 4000: 6249.494629
Minibatch accuracy: 62.5%
Validation accuracy: 80.0%
Minibatch loss at step 4500: 5640.530762
Minibatch accuracy: 63.3%
Validation accuracy: 79.0%
Minibatch loss at step 5000: 5447.071289
Minib

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
